In [ ]:
PATH = "/home/xjaros2/Documents/git/csidh-setup/"
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'

SS_VER = 'SS_VER_2_1'
attack_type = "A1"

In [ ]:
%cd $PATH/scripts
%run ./init.ipynb
%cd $PATH/scripts

In [ ]:
# csidh.voltage_glitching_setup()
csidh.scope.cglitch_setup()

In [ ]:
Unit.is_husky = csidh.scope._is_husky

if Unit.is_husky:
    Unit.OFFSET_MIN = 0 
    Unit.OFFSET_MAX = csidh.scope.glitch.phase_shift_steps // 2 	
    Unit.OFFSET_RANGE = Unit.OFFSET_MAX - Unit.OFFSET_MIN
    
    Unit.WIDTH_MIN = 0
    Unit.WIDTH_MAX = csidh.scope.glitch.phase_shift_steps // 2
    Unit.WIDTH_RANGE = Unit.WIDTH_MAX - Unit.WIDTH_MIN
else:
    Unit.OFFSET_MIN = -48 	 
    Unit.OFFSET_MAX = 48
    Unit.OFFSET_RANGE = Unit.OFFSET_MAX - Unit.OFFSET_MIN
    
    Unit.WIDTH_MIN = 0
    Unit.WIDTH_MAX = 48
    Unit.WIDTH_RANGE = Unit.WIDTH_MAX - Unit.WIDTH_MIN

Unit.EXT_OFFSET_MIN = 0
Unit.EXT_OFFSET_MAX = MAX_EXT_OFFSET // 4 
Unit.EXT_OFFSET_RANGE = Unit.EXT_OFFSET_MAX - Unit.EXT_OFFSET_MIN

Unit.REPEAT_MIN = 1
Unit.REPEAT_MAX = 10

In [ ]:
csidh.action_sleep = 0.25

In [ ]:
POPULATION_SIZE = 250000
CACHE_INTERVAL = 2
NUM_TRIES = 1

cache.clear()

def evaluate_unit_wrapper(csidh, unit):
    evaluate_unit(csidh, unit, NUM_TRIES)

population = generate_population(POPULATION_SIZE)
for i in trange(0, len(population), CACHE_INTERVAL):
    batch = population[i:i+CACHE_INTERVAL]
    evaluate_batch(csidh, batch, evaluate_unit_wrapper)
    write_cache_to_file(f"husky-clock-param-search-random-(+5)(-3)(+1).json", cache)  